In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
import datetime

np.random.seed(42)

import sys
print(sys.executable) # just to check which python

import gym
from gym import spaces

from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

/usr/local/opt/python@3.9/bin/python3.9


In [2]:
class RequestType:
    def __init__(self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.bw = bandwidth
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate
        
        self.num_made = 0
        self.num_accepted = 0

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None, parent_elastic=None, bw_dist=None, request_type_template=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        self.parent_elastic = parent_elastic
        self.accepted = None
        self.path = None
        self.bw_dist = bw_dist
        
        self.blueprint = request_type_template
        
        if request_type == "elastic":
            self.distribution = distribution
            self.scale_requests = []
                   
        if request_type_template is not None:
            request_type_template.num_made += 1
            
    def add_scale_request(self, req): 
        # we store related scale requests for elastic requests
        # not used if static request
        self.scale_requests.append(req)
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment)).flatten()
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment)).flatten()
    
        if self.request_type == "static":
            one_hot_type = torch.tensor([1, 0, 0])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([0, 1, 0])
        elif self.request_type == "scale":
            one_hot_type = torch.tensor([0, 0, 1])
            
        encoding = torch.cat([one_hot_source, 
                             one_hot_dest,
                             torch.tensor([self.bw]), 
                             torch.tensor([self.service_time]),
                             one_hot_type])
        
        return encoding

In [3]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remove_request(self, request_obj):
        self.serving_requests.remove(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [4]:
class Environment(gym.Env):
    # requests_in_service_encoder = nn.RNN(????, 7)
    metadata = {'render.modes': ['human']}
    
    def __init__(self, nodes, links, request_blueprints, use_RNN=False, sb3_compat=False):
        super(Environment, self).__init__()
                
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.request_blueprints = request_blueprints
        self.last_time = 0
        self.episode_timesteps = 600
        self.use_RNN = use_RNN
        self.sb3_compat = sb3_compat
        self.precomputed_paths = {}
        
        if sb3_compat:
            self.request_being_considered = None
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
        
        # Setup gym-specific code
        env_encoding_size = self.get_encoding(increment_iterator=False).size()
        req_encoding_size = self.request_list[0].get_encoding(self.nodes).size()
        # print(env_encoding_size[0] + req_encoding_size[0])
        
        self.action_space = spaces.Box(low=0, high=math.inf,
                                      shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-math.inf, high=math.inf,
                                      shape=(env_encoding_size[0] + req_encoding_size[0],), dtype=np.float32)
        
        # TODO, WRITE RNN logic
        
        #if use_RNN:
        #    self.requests_in_service_encoder = nn.RNN
        
    def precompute_paths(self):
        for req_type in self.request_blueprints:
            self.precomputed_paths[req_type.source + req_type.sink] = env.search(req_type.source, req_type.sink, [], [])
            
    def add_request(self, request, path=None): # we want to add this request to a link or path
        # path: a list of nodes that the request traverses including source and sink
        # if no path is specified, path is assumed to be [req.source, req.sink]
        
        if path is not None: 
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                env.links[node_pair[0] + node_pair[1]].add_request(request)
        
        else:
            self.links[request.source + request.sink].add_request(request)
        
        request.accepted = True
        request.blueprint.num_accepted += 1
        self.request_history.append(request)
        # print(self.links[request.source + request.sink])
    
    def reset(self):
        for link in self.links.values():
            link.reset()
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.last_time = 0
        self.request_list = env.create_requests()
        self.request_queue = iter(self.request_list)
        
        return env.get_encoding()
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        bw = request.bw
        if request.type == "elastic":
            #base_rate = request.bw
            type_bonus = 1.1                # 1.1 when elastic
            bw = np.array(request.bw_dist).dot(request.distribution)
            
        r = bw * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link(s) < 0, very "bad" reward
        if request.path is not None:
            path_length = len(request.path)
            
            r *= math.pow(0.9, path_length - 2)
            
            nodes = [[request.path[i], request.path[i + 1]] for i in range(len(request.path) - 1)]
            for node_pair in nodes:
                if self.links[node_pair[0] + node_pair[1]].remaining_bw() < 0:
                    print("exceeded: " + str(-r * 10))
                    return (-r * 10)
        else:
            # path is direct, so no decrease of reward needed
            remaining_bw = self.links[request.source + request.sink].remaining_bw()
            if remaining_bw < 0:
                print("exceeded: " + str(-r * 10))
                return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static" or request.type == "elastic":
                return 0
            elif request.type == "scale":
                if len(self.past_distributions) == 0:
                    return -1 * r
                
                else:
                    current_sum = torch.from_numpy(np.sum(self.past_distributions, axis=0))

                    average_past_distribution = current_sum / len(self.past_distributions)
                    current_req_distribution = torch.tensor(request.parent_elastic.distribution)

                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))

                """
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                """
                
    def next_req(self):
        next_req = next(self.request_queue)
        if self.sb3_compat:
            self.request_being_considered = next_req
        return next_req
                
    def step(self, action, req=None):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        # this is given by our agent
        
        # if req is None, that means we are using sb3_compat=True and we can get the req from self.request_being_considered
                
        if req is None:
            req = self.request_being_considered
            
        if action[0] > 0.5:
            # accept request
            paths = env.precomputed_paths[req.source + req.sink]
            paths.sort(key=lambda x: len(x)) # sort by shortest path
            # select the path we are using
            path = paths[action[1:4].argmax()]
            
            self.add_request(req, path)
        
            reward = env.reward(req, "accept")
        elif action[0] < 0.5:
            # reject
            reward = env.reward(req, "reject")
        
        obs = env.get_encoding()
        
        done = req.arrival_time > 600
        info = {}
        
        return obs, reward, done, info
        
    def update_requests(self, current_time):
        # here, we remove expired requests and update E_history based off of the request stats
        
        for link in self.links.values():
            for request in link.serving_requests.copy():
                if (request.arrival_time + request.service_time) > self.last_time and (request.arrival_time + request.service_time) < current_time:
                    # request has expired, let's remove it from the links
                    for link in self.links.values():
                        if request in link.serving_requests:
                            link.remove_request(request)

                    if request.type == "elastic":
                        time_on_higher_bw = 0
                        for scale_req in request.scale_requests:
                            time_on_higher_bw += scale_req.service_time

                        time_on_lower_bw = request.service_time - time_on_higher_bw

                        # calculate E[history]
                        request_time = np.array([time_on_lower_bw, time_on_higher_bw])
                        request_bw = request.bw
                        result = (request_time / request_time.sum()).dot(request_bw)
                        self.past_distributions.append(request_time / request_time.sum())
                        self.E_history.append(result)

    def get_encoding(self, increment_iterator=True):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        if increment_iterator:
            next_req = self.next_req()

            while next_req.type == "scale":
                if not next_req.parent_elastic.accepted:
                    next_req = self.next_req()
                elif next_req.parent_elastic.accepted:
                    break
            
            """
            while next_req.type == "scale":
                if next_req.parent_elastic.accepted:
                    next_req.accepted = True # we must accept since we accepted elastic req
                    self.add_request(next_req, next_req.parent_elastic.path)
                next_req = self.next_req()
            """
            self.update_requests(next_req.arrival_time)

        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        if not increment_iterator:
            return torch.tensor(env_encoding)
        
        if self.sb3_compat:
            return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])
        else:
            return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req
    
    def create_requests(self):
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
        
            while last_arrival < self.episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, 
                                      request_type.bw[0], request_type.distribution, bw_dist=request_type.bw, request_type_template=request_type)
                requests.append(new_request)
                
                if request_type.type == "elastic": 
                    # we will start with the first bandwidth element as starting bw
                    # WE ASSUME that bw[0] < bw[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.bw[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.bw[0]:
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.bw[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[1])
                            scale_request = Request("scale", scale_service_time, \
                                                    arrival_time + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw, parent_elastic=new_request,
                                                   request_type_template=request_type)
                            requests.append(scale_request)
                            new_request.add_scale_request(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.bw[1] # request_type.bw[0] + scale_bw
                        elif current_bw == request_type.bw[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[0])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.bw[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests
    
    def search(self, source, dest, visited_a, paths):
        visited_a.append(source)
        # print(visited_a)

        for link in set(env.links.values()):
            visited = visited_a.copy()
            if source in link.nodes:
                if dest in link.nodes:
                    visited.append(dest)
                    paths.append(visited)

                x = link.nodes.copy()
                x.remove(source)
                if x[0] not in visited:
                    self.search(x[0], dest, visited.copy(), paths)
        return paths
    
    def print_statistics(self):
        for req_type in self.request_blueprints:
            print(req_type.source + " | " +
                   req_type.sink + " | " + 
                 "BW: " + str(req_type.bw) + " | " +
                 "Arrival rate: " + str(req_type.arrival_rate) + " | " +
                 "Acceptance rate: " + str(req_type.num_accepted / req_type.num_made))
            
        resources_used = 0
        for req in self.request_list:
            if req.accepted == True:
                resources_used += req.bw * req.service_time
            
        print("BW used: " + str(resources_used))

In [5]:
env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                   ["c", "d", 20], ["c", "e", 10], ["d", "f", 10], \
                                                   ["e", "f", 10]], \
                  [RequestType("static", [2], 0.5, 0.75, "a", "b", [1]), \
                  RequestType("static", [8], 1, 1.5, "a", "b", [1]), \
                  RequestType("elastic", [4, 9], 1, 1.5, "a", "b", [0.8, 0.2], switch_rate=[0.08, 0.02]), \
                  RequestType("static", [1], 1, 1.5, "c", "d", [1]), \
                  RequestType("static", [7], 0.5, 0.75, "c", "d", [1]), \
                  RequestType("elastic", [3, 13], 2, 3, "c", "d", [0.9, 0.1], switch_rate=[0.09, 0.01]), \
                  RequestType("static", [3], 0.5, 0.75, "e", "f", [1]), \
                   RequestType("static", [6], 1, 1.5, "e", "f", [1]), \
                    RequestType("elastic", [5, 8], 2, 3, "e", "f", [0.7, 0.3], switch_rate=[0.07, 0.03])],
                 sb3_compat=True)


                # self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None
    
env.precompute_paths()

### Choose shortest viable path

In [6]:
def policy(env_encoding, next_req_encoding, next_req_obj):    
    # find all paths between source and sink
    paths = env.precomputed_paths[next_req_obj.source + next_req_obj.sink] 
    # (env.search(next_req_obj.source, next_req_obj.sink, [], []))
    paths.sort(key=lambda x: len(x)) # sort by shortest path
    selection = 0
    for path in paths:
        # check if this path works
        works = True
        nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
        for node_pair in nodes:
            if env.links[node_pair[0] + node_pair[1]].remaining_bw() < next_req_obj.bw:
                works = False
                
        if works:
            selection = paths.index(path)
            selection_one_hot = nn.functional.one_hot(torch.tensor([selection]), num_classes=3).flatten()
            next_req_obj.path = path
            return torch.cat([torch.tensor([1]), selection_one_hot])
        
    return torch.cat([torch.tensor([0]), torch.tensor([0,0,0])])

In [7]:
# ONLY WORKS WITH sb3_compat=False when creating env object

total_reward = 0
env_encoding, next_req_encoding, next_req_obj = env.reset()
done = False

while not done:
    decision = policy(env_encoding, next_req_encoding, next_req_obj)
    
    obs, reward, done, info = env.step(decision, next_req_obj)
    env_encoding, next_req_encoding, next_req_obj = obs
    
    total_reward += reward

print(total_reward)

<ipython-input-4-3a01dc88a67e>:276: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


11929.026038920432


In [8]:
env.print_statistics()

a | b | BW: [2] | Arrival rate: 0.75 | Acceptance rate: 0.4870530209617756
a | b | BW: [8] | Arrival rate: 1.5 | Acceptance rate: 0.26838709677419353
a | b | BW: [4, 9] | Arrival rate: 1.5 | Acceptance rate: 0.3286232950642724
c | d | BW: [1] | Arrival rate: 1.5 | Acceptance rate: 0.5038659793814433
c | d | BW: [7] | Arrival rate: 0.75 | Acceptance rate: 0.38267148014440433
c | d | BW: [3, 13] | Arrival rate: 3 | Acceptance rate: 0.23015406327178642
e | f | BW: [3] | Arrival rate: 0.75 | Acceptance rate: 0.46183699870633893
e | f | BW: [6] | Arrival rate: 1.5 | Acceptance rate: 0.3296432964329643
e | f | BW: [5, 8] | Arrival rate: 3 | Acceptance rate: 0.3038709677419355
BW used: 12511.685723575798


### PPO

In [6]:
model = PPO(MlpPolicy, env, verbose=1, tensorboard_log="./multilink/")
model.learn(total_timesteps=600000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./multilink/PPO_1


<ipython-input-4-3a01dc88a67e>:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


exceeded: -62.715225413816114
exceeded: -22.424637363251733
exceeded: -0.12332758318444846
exceeded: -0.3450352357883517
exceeded: -0.6901326401488881
exceeded: -0.5279052227521029
exceeded: -0.4133610853583354
exceeded: -1.146456680822214
exceeded: -0.38904006699889715
exceeded: -4.214889860839603
exceeded: -12.625222562148
exceeded: -84.67413229508406
exceeded: -0.6491359368342537
exceeded: -4.834806473854401
exceeded: -22.052043944613576
exceeded: -22.584472706306833
exceeded: -8.072674616543821
exceeded: -110.08035188628551
exceeded: -3.292313398885808
exceeded: -19.771028245734414
exceeded: -2.1276617118515477
exceeded: -87.90445605214487
exceeded: -49.47555053300884
exceeded: -2.8088246771416507
exceeded: -33.659249112903126
exceeded: -331.4671474855761
exceeded: -0.09548062120440502
exceeded: -0.2117843945972941
exceeded: -0.2821957916096936
exceeded: -1.5238974218368
exceeded: -2.249007310992488
exceeded: -33.130551137988164
exceeded: -0.8817413470068909
exceeded: -7.1211914306

exceeded: -0.6986167230869702
exceeded: -0.001582432533624942
exceeded: -0.20367574859874463
exceeded: -5.183563998268359
exceeded: -0.8936690552156161
exceeded: -0.1919641568741627
exceeded: -0.20497515856084875
exceeded: -1.7740635722021392
exceeded: -28.121764986444635
exceeded: -0.5291415976009344
exceeded: -20.503488435368258
exceeded: -122.65944171314874
exceeded: -46.99809122965917
exceeded: -2.046137297366246
exceeded: -136.55598700152007
exceeded: -0.9324095306159232
exceeded: -0.5840975473288553
exceeded: -3.1892503393615095
exceeded: -0.41484272458489624
exceeded: -0.8405510622605016
exceeded: -0.12829503161128092
exceeded: -2.7225008528611854
exceeded: -0.6269693065815618
exceeded: -0.029546118166995393
exceeded: -1.155830798807151
exceeded: -0.44673454413942737
exceeded: -0.5274901707757201
exceeded: -0.4921643412203691
exceeded: -0.9077166765054099
exceeded: -1.2654820873080117
exceeded: -0.4285446794721681
exceeded: -1.2191512319257232
exceeded: -1.6376681600215934
excee

exceeded: -0.6954715942762651
exceeded: -1.3856071372875323
exceeded: -0.07643569125611355
exceeded: -0.353859457531601
exceeded: -10.383582793786406
exceeded: -19.732023624362604
exceeded: -24.409386443354965
exceeded: -7.2020534135370475
exceeded: -5.31336378827573
exceeded: -6.658071599020127
exceeded: -85.99838515660369
exceeded: -79.08714988041719
exceeded: -0.9155698965973276
exceeded: -2.1226304013536255
exceeded: -77.16808624485722
exceeded: -2.6049848093532924
exceeded: -0.16678823646013946
exceeded: -0.1735045823278361
exceeded: -0.6084642928095233
exceeded: -0.25426471649850824
exceeded: -0.07089190972241008
exceeded: -0.08237798169508534
exceeded: -1.0445521759835987
exceeded: -19.804582368581233
exceeded: -2.1372941062212307
exceeded: -0.29608106097064046
exceeded: -14.999190734725815
exceeded: -1.5697410486190977
exceeded: -1.2132983088061484
exceeded: -0.320236224343344
exceeded: -0.226535649531191
exceeded: -14.005739852843726
exceeded: -3.022925860504553
exceeded: -0.4

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.17e+04    |
|    ep_rew_mean          | 2.6e+03     |
| time/                   |             |
|    fps                  | 1095        |
|    iterations           | 270         |
|    time_elapsed         | 504         |
|    total_timesteps      | 552960      |
| train/                  |             |
|    approx_kl            | 0.027373577 |
|    clip_fraction        | 0.221       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.34       |
|    explained_variance   | 0.673       |
|    learning_rate        | 0.0003      |
|    loss                 | 39          |
|    n_updates            | 2690        |
|    policy_gradient_loss | -0.0137     |
|    std                  | 0.579       |
|    value_loss           | 77.2        |
-----------------------------------------
exceeded: -1.0416938004743463
exceeded: -0.09630043997411653
exceeded: -0.32

exceeded: -10.800634352086394
exceeded: -0.17859339987543188
exceeded: -7.091782223036869
exceeded: -116.59262367372273
exceeded: -1.7959983920492495
exceeded: -0.32100165029449373
exceeded: -0.4320648926664281
exceeded: -0.004085344475589739
exceeded: -45.9281065895511
exceeded: -16.996008728783853
exceeded: -37.03738158686852
exceeded: -10.658418977369832
exceeded: -1.824517626887269
exceeded: -13.40310821108796
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.18e+04    |
|    ep_rew_mean          | 2.66e+03    |
| time/                   |             |
|    fps                  | 1097        |
|    iterations           | 276         |
|    time_elapsed         | 515         |
|    total_timesteps      | 565248      |
| train/                  |             |
|    approx_kl            | 0.031873755 |
|    clip_fraction        | 0.272       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.32   

exceeded: -10.198082014017942
exceeded: -1.3525306154152237
exceeded: -0.13928720510841464
exceeded: -2.4919377601319352
exceeded: -0.13095723094502945
exceeded: -3.4643372738884075
exceeded: -2.1358657120864692
exceeded: -1.1350430473074038
exceeded: -0.44745262187785134
exceeded: -81.31080092995809
exceeded: -12.744151401707871
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.18e+04    |
|    ep_rew_mean          | 2.66e+03    |
| time/                   |             |
|    fps                  | 1095        |
|    iterations           | 280         |
|    time_elapsed         | 523         |
|    total_timesteps      | 573440      |
| train/                  |             |
|    approx_kl            | 0.018475082 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.31       |
|    explained_variance   | 0.251       |
|    learning_rate        | 0.0003    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.18e+04    |
|    ep_rew_mean          | 2.68e+03    |
| time/                   |             |
|    fps                  | 1095        |
|    iterations           | 285         |
|    time_elapsed         | 532         |
|    total_timesteps      | 583680      |
| train/                  |             |
|    approx_kl            | 0.019576252 |
|    clip_fraction        | 0.274       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.24       |
|    explained_variance   | 0.314       |
|    learning_rate        | 0.0003      |
|    loss                 | 23.2        |
|    n_updates            | 2840        |
|    policy_gradient_loss | -0.0087     |
|    std                  | 0.567       |
|    value_loss           | 57.4        |
-----------------------------------------
exceeded: -143.86829085844172
exceeded: -0.21511014985575058
exceeded: -6.36

exceeded: -104.5806473158205
exceeded: -0.5020633847981257
exceeded: -0.9849018443477753
exceeded: -1.6233285930403254
exceeded: -0.136318305400963
exceeded: -0.3771098094981884
exceeded: -0.0751728111796877
exceeded: -2.9728392765265284
exceeded: -0.8920757681507586
exceeded: -1.1257007328734994
exceeded: -1.6656291809607673
exceeded: -0.3520553507219826
exceeded: -0.17671463176730348
exceeded: -23.62754503121358
exceeded: -0.21054262372204838
exceeded: -0.29267218477324514
exceeded: -1.1728125426751321
exceeded: -15.502582342370733
exceeded: -0.2886866275803899
exceeded: -0.2747237040567503
exceeded: -58.01586211572478
exceeded: -2.669038073146803
exceeded: -21.923478381521537
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.18e+04  |
|    ep_rew_mean          | 2.73e+03  |
| time/                   |           |
|    fps                  | 1095      |
|    iterations           | 291       |
|    time_elapsed         | 543 

In [8]:
total_reward = 0
observation = env.reset()
done = False
while not done:
    action, _states = model.predict(observation)
        
    observation, reward, done, info = env.step(action)
        
    total_reward += reward
    
print(total_reward)

<ipython-input-4-3a01dc88a67e>:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])


exceeded: -4.665130686013766
exceeded: -1.9536245346367898
exceeded: -7.877867497882596
exceeded: -192.71008086654976
exceeded: -1.5878856665312315
exceeded: -0.1412544469174888
exceeded: -0.2424055613866777
exceeded: -0.6162871322937709
exceeded: -2.208778564138499
exceeded: -0.18086723527172235
exceeded: -0.8025489212264398
exceeded: -0.4603388968658789
exceeded: -0.15715175486795135
exceeded: -2.219888236372896
exceeded: -0.5478824084612736
exceeded: -6.347802176238988
exceeded: -0.8073332115947769
exceeded: -0.5928143275961243
exceeded: -0.950266292882262
exceeded: -0.21829156851494824
exceeded: -26.79816321464013
exceeded: -5.578838775822996
exceeded: -0.4248945799279231
exceeded: -0.4880356308259536
exceeded: -0.2470708113930435
exceeded: -0.35435324953108827
exceeded: -0.6967848070966103
exceeded: -0.08367250716870611
exceeded: -0.8187467048732394
exceeded: -0.06905973139019997
exceeded: -1.2733695278473525
exceeded: -0.7335783723175836
exceeded: -0.504025034678973
exceeded: -0.

In [ ]:
env.print_statistics()

In [9]:
model.learn(total_timesteps=6000000, reset_num_timesteps=False)

Logging to ./multilink/PPO_1
exceeded: -0.7130045855659766


<ipython-input-4-3a01dc88a67e>:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])


exceeded: -1.3580365860941266
exceeded: -104.54600831189973
exceeded: -5.2336595267987285
exceeded: -1.8480526945021147
exceeded: -41.469908927527435
exceeded: -61.47097638466665
exceeded: -2.6656034657694496
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.24e+04 |
|    ep_rew_mean     | 5.09e+03 |
| time/              |          |
|    fps             | 1503088  |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1802240  |
---------------------------------
exceeded: -1.25913437334013
exceeded: -0.474073222378072
exceeded: -7.724811168411957
exceeded: -1.8112713474509479
exceeded: -61.150143749719376
exceeded: -2.9456761703297945
exceeded: -0.4048069921136552
exceeded: -53.078010767226395
exceeded: -66.09842449695746
exceeded: -0.8066890364038517
exceeded: -0.5007794425420568
exceeded: -0.18517936080109001
exceeded: -189.06952276250257
exceeded: -0.9777572217914401
exceeded: -2.194525214265414
exceed

exceeded: -53.19841861387957
exceeded: -6.625230452081237
exceeded: -0.818083675473589
exceeded: -2.669695493230929
exceeded: -3.351282985038844
exceeded: -15.730492543216425
exceeded: -53.924626211286565
exceeded: -0.030834729533768562
exceeded: -15.066694547362223
exceeded: -0.3118603559079851
exceeded: -0.40436596304487404
exceeded: -1.9286910261518426
exceeded: -0.42958985615011147
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.24e+04    |
|    ep_rew_mean          | 5.12e+03    |
| time/                   |             |
|    fps                  | 122021      |
|    iterations           | 8           |
|    time_elapsed         | 14          |
|    total_timesteps      | 1816576     |
| train/                  |             |
|    approx_kl            | 0.054834764 |
|    clip_fraction        | 0.431       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.913       |
|    explained_variance

exceeded: -7.7283938772328735
exceeded: -146.5488243760577
exceeded: -9.199416015102242
exceeded: -3.4982087234123123
exceeded: -35.08083119420834
exceeded: -20.192663318176347
exceeded: -0.43137048291241775
exceeded: -30.28921651543596
exceeded: -1.327422778462191
exceeded: -4.836986539643281
exceeded: -21.569443521704596
exceeded: -2.957051180483639
exceeded: -35.2259815749584
exceeded: -2.849043024177158
exceeded: -12.354387399954422
exceeded: -4.340942795739472
exceeded: -0.3886030120799343
exceeded: -0.42078949055920306
exceeded: -2.0943356378198876
exceeded: -0.9043590055722943
exceeded: -0.5301852436802424
exceeded: -1.035529481641797
exceeded: -0.31828016619501753
exceeded: -1.05577435230524
exceeded: -0.13774356988791225
exceeded: -1.5576590639765961
exceeded: -107.18023932032003
exceeded: -0.3828482827178101
exceeded: -0.43115151176458155
exceeded: -2.4041985073448706
exceeded: -0.7519212023398452
----------------------------------------
| rollout/                |           

...

exceeded: -21.50259539885955
exceeded: -4.4183638497808335
exceeded: -8.048129202844121
exceeded: -32.87231660735354
exceeded: -37.37172310391927
exceeded: -4.362152862953346
exceeded: -0.0038603781008240464
exceeded: -13.100788544037375
exceeded: -0.1410102061099264
exceeded: -57.07634834525357
exceeded: -1.156775547704022
exceeded: -18.180278074212907
exceeded: -0.009857908833886386
exceeded: -28.744150654728415
exceeded: -0.4878647908633119
exceeded: -11.188797576943797
exceeded: -0.07498987966512069
exceeded: -27.322313469241195
exceeded: -3.523832702107439
exceeded: -7.705510013624018
exceeded: -0.023813435036009124
exceeded: -0.07943610287803818
exceeded: -1.1426850114879044
exceeded: -0.24191007580600976
exceeded: -14.360044252519952
exceeded: -0.11586419902655783
exceeded: -1.3795629282078115
exceeded: -0.37104462526089066
exceeded: -0.6313637298612931
exceeded: -1.064725579501157
exceeded: -40.33767613859087
exceeded: -34.3966653068878
-----------------------------------------

exceeded: -0.8286670473830466
exceeded: -26.57008765751907
exceeded: -0.21901934689178298
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.14e+04   |
|    ep_rew_mean          | 5.24e+03   |
| time/                   |            |
|    fps                  | 2939       |
|    iterations           | 577        |
|    time_elapsed         | 1014       |
|    total_timesteps      | 2981888    |
| train/                  |            |
|    approx_kl            | 0.11664106 |
|    clip_fraction        | 0.563      |
|    clip_range           | 0.2        |
|    entropy_loss         | 1.71       |
|    explained_variance   | 0.301      |
|    learning_rate        | 0.0003     |
|    loss                 | 21.5       |
|    n_updates            | 14550      |
|    policy_gradient_loss | 0.0088     |
|    std                  | 0.16       |
|    value_loss           | 82.6       |
----------------------------------------
exceeded

exceeded: -0.8079441689787195
exceeded: -0.23028925114292537
exceeded: -1.3127338880158013
exceeded: -0.15289009314195068
exceeded: -0.04443659785317789
exceeded: -64.62211778202405
exceeded: -207.5355780431282
exceeded: -2.7516806913041303
exceeded: -0.43340945732342206
exceeded: -1.9495790268578974
exceeded: -4.968010757760755
exceeded: -0.0591541661040532
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.14e+04    |
|    ep_rew_mean          | 5.24e+03    |
| time/                   |             |
|    fps                  | 2927        |
|    iterations           | 581         |
|    time_elapsed         | 1021        |
|    total_timesteps      | 2990080     |
| train/                  |             |
|    approx_kl            | 0.057329718 |
|    clip_fraction        | 0.355       |
|    clip_range           | 0.2         |
|    entropy_loss         | 1.7         |
|    explained_variance   | 0.263       |
|    lear

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.14e+04    |
|    ep_rew_mean          | 5.24e+03    |
| time/                   |             |
|    fps                  | 2919        |
|    iterations           | 584         |
|    time_elapsed         | 1026        |
|    total_timesteps      | 2996224     |
| train/                  |             |
|    approx_kl            | 0.054764356 |
|    clip_fraction        | 0.382       |
|    clip_range           | 0.2         |
|    entropy_loss         | 1.72        |
|    explained_variance   | 0.693       |
|    learning_rate        | 0.0003      |
|    loss                 | 159         |
|    n_updates            | 14620       |
|    policy_gradient_loss | -0.0111     |
|    std                  | 0.159       |
|    value_loss           | 210         |
-----------------------------------------
exceeded: -6.185179722370544
exceeded: -2.0298609297102645
exceeded: -0.3312

KeyboardInterrupt: 

In [10]:
model.save("3M_trained")

In [ ]:
today()